In [1]:
import pandas as pd
import numpy as np
from clickhouse_driver import Client
import networkx as nx
import re
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

In [2]:
# 读取npm包相关的数据
file_path = "C:/Users/lenovo/Desktop/2024-05-13-16-54-33_EXPORT_CSV_13529018_459_0.csv"
npm_packages_df = pd.read_csv(file_path)
npm_packages_df = npm_packages_df.dropna()

In [3]:
## 提取出上述url的owner和repo名对（用以后续爬取贡献信息）
def get_owner_repo(url):
    # 定义正则表达式模式
    pattern = r'\/([^\/]+)\/([^\/]+)\.git(?:\/|$)'
    match = re.search(pattern, url)
    if match:
        # o = match.group(1)
        r = match.group(2)
        # return o, r       ## 可以返回出来（调用函数时，申明这对变量）
        return r
    else:
        return None, None

In [4]:
## 取得每一行的repo，变成数据框的一整列
npm_packages_df["repo_name"] = npm_packages_df["repository_url"].apply(get_owner_repo)
print(npm_packages_df)

                                   package_id                          name  \
0        67d418e0-5926-43a2-bf77-d8933f0e70ab                 @minta/logger   
2        1e8f64b6-6252-46ab-bd81-8225986bf593         @mintapp/ng-bootstrap   
4        f9ba3988-cf09-4c36-bae3-29b7829ac85a          @mintblue/sdk-server   
5        a76da626-6142-40fd-bdac-00490cf773cb  @mintcity/cardano-dapp-utils   
6        6cf7ab27-0e79-4f2b-bcfb-ac8988ad6cee       @mintdevops/odyssey-cli   
...                                       ...                           ...   
999993   9dc27c82-a43c-4bf0-b969-1b9a06f3fa2f              littlebro-client   
999995   0a39f21f-bcff-4ce2-8d16-a487d1b66e55               littlecrocodile   
999996   f7c074bf-0314-40fd-be32-6e3fab0fbca4                      littledb   
999999   fd05f534-c0d8-4bbb-b326-24d3754ce099                  littlefinger   
1000000  f0a51aef-a14b-4903-bed7-8ec61b410fa9                    littlefoot   

                 version                           

In [5]:
# 读取repo_co相关的数据
repo_co_df = pd.read_csv("contributors_of_repo.csv")

In [8]:
## 现在我们有了repo_co_df和npm_packages_df两个数据框，他们具有共同的字段repo_name
## 因此，我们实际上可以使用类似于数据库的JOIN算子，对他们进行连接，从而形成更大的数据集
    ## 我们想从repo_co映射到npm，所以repo_co作为左表，npm作为右表
# result = repo_co_df.join(npm_packages_df)
reflection_of_repo_to_npm = pd.merge(repo_co_df, npm_packages_df, on="repo_name", how='inner')
reflection_of_repo_to_npm

,repo_name,developers,package_id,name,version,description,repository_type,repository_url,license,homepage,time
0,ng-bootstrap,"['maxokorokov', 'pkozlowski-opensource', 'jniz...",1e8f64b6-6252-46ab-bd81-8225986bf593,@mintapp/ng-bootstrap,1.0.0-beta.6-v3,Angular powered Bootstrap,git,git+https://github.com/ng-bootstrap/ng-bootstr...,[object Object],https://github.com/ng-bootstrap/ng-bootstrap#r...,2018-10-09 10:56:10
1,ng-bootstrap,"['maxokorokov', 'pkozlowski-opensource', 'jniz...",7691ce0e-072b-49a6-bf81-93cfe07d7d6c,@ng-bootstrap/ng-bootstrap,9.1.3,Angular powered Bootstrap,git,git+https://github.com/ng-bootstrap/ng-bootstr...,[object Object],https://github.com/ng-bootstrap/ng-bootstrap#r...,2022-06-14 12:00:39
2,ng-bootstrap,"['maxokorokov', 'pkozlowski-opensource', 'jniz...",f30e25f8-5ddd-4731-b762-44549bc786d1,@anglr/bootstrap,7.0.1-beta.20200304081421,Angular module wrapping bootstrap components,git,git+https://github.com/ressurectit/ng-bootstra...,MIT,https://github.com/ressurectit/ng-bootstrap,2020-03-04 08:14:23
3,ng-bootstrap,"['maxokorokov', 'pkozlowski-opensource', 'jniz...",7816355d-2b6e-4136-8d36-f35ebbb5127f,@rajkeshwar/ng2v-components,1.0.1,ng2v-components powered by ng-bootstrap,git,git+https://github.com/ng-bootstrap/ng-bootstr...,[object Object],https://github.com/ng-bootstrap/ng-bootstrap#r...,2017-05-22 08:44:34
4,ng-bootstrap,"['maxokorokov', 'pkozlowski-opensource', 'jniz...",092e6caf-b871-4557-9161-40d7f213204f,@rajkeshwar/ng-bootstrap,1.0.0-alpha.25,Angular 2 powered Bootstrap,git,git+https://github.com/ng-bootstrap/ng-bootstr...,[object Object],https://github.com/ng-bootstrap/ng-bootstrap#r...,2017-05-05 03:01:41
...,...,...,...,...,...,...,...,...,...,...,...
4777567,little-imgui,['TheBunyip'],1ac997e5-9b22-4546-bb4d-8eed81c46663,little-imgui,1.0.6,Immediate GUI inspired by Dear IMGUI,git,git+ssh://git@github.com/TheBunyip/little-imgu...,ISC,https://github.com/TheBunyip/little-imgui/tree...,2022-12-05 11:36:01
4777568,little-insomniac,['ianwieds'],b83c0d76-793e-4a4a-b9f9-77c9224ba337,little-insomniac,0.0.7,Create custom text snippets that you can acces...,git,git+https://github.com/little-insomniac/little...,MIT,https://little-insomniac.com,2022-07-15 23:22:48
4777569,little-template,['maskedcoder'],6fbac95f-0b17-4104-9cdd-0cd0d3642479,little-template,1.0.0,A simple HTML template engine,git,https://github.com/maskedcoder/little-template...,MIT,https://github.com/maskedcoder/little-template,2015-05-05 19:14:40
4777570,littlebad-bot,['clover14580'],ff8602d2-c72e-42ef-b17b-4e6232a8d1bb,littlebad-bot,1.0.0,基于 [oicq](https://github.com/takayama-lily/oic...,git,git+https://github.com/LittleBadBad/littlebad-...,MIT,https://github.com/LittleBadBad/littlebad-bot#...,2022-09-19 06:07:47


In [7]:
## 数据集reflection_of_npm_and_repo则表示了从repo到npm的映射
reflection_of_repo_to_npm
reflection_of_repo_to_npm.to_csv("reflection_of_repo_to_npm.csv")

KeyboardInterrupt: 